# **Basic But Dumb Algorithm**
Why is it dumb? Uhm.. I'll answer it later

In [102]:
import random
import pandas as pd

In [103]:
content_categories = ["cat", "dog", "space"]

In [104]:
columns = ["video_id", "content_category", "like", "score" ]

In [105]:
content = []

#lets generate some content, in real world this is generated by the users
for i in range(100):
    video_id = f"video_{i}"
    content_category = content_categories[random.randint(0,2)] #for now we assume we already know "what" the content it about.
    like = 0 #0 indicates it is not liked by user
    score = like #if liked, then score is updated
    content.append([video_id, content_category, like, score ])


In [106]:
df = pd.DataFrame(data=content, columns=columns)
df


,video_id,content_category,like,score
0,video_0,cat,0,0
1,video_1,cat,0,0
2,video_2,space,0,0
3,video_3,space,0,0
4,video_4,cat,0,0
...,...,...,...,...
95,video_95,cat,0,0
96,video_96,space,0,0
97,video_97,dog,0,0
98,video_98,cat,0,0


In [107]:
#now lets try to make a very rudimentary system to mimic the interaction
#if you create a new account on any social media it will show the most popular content curated by mods or will throw random content at you.
#and then it will learn based on how you interact. 
#for simplicity we are going to use random content strategy. 
logs =[] #to keep a track of user's preferences

while True:
    len_content = len(df)
    random_int = random.randint(0, len_content-1)
    
    display_content_dict = df.iloc[random_int].to_dict()
    if display_content_dict["like"] == 0:
        response = input(f"Current Video: {display_content_dict['video_id']} is about {display_content_dict['content_category']}.\n Press y to like, or click enter to see new content")
        if response == "y":
            df.at[random_int, "like"] = 1
            df.at[random_int, "score"] += 1
            print(f"You liked {display_content_dict['video_id']} about {display_content_dict['content_category']}")
            print( df.iloc[random_int].to_dict())
            logs.append(df.iloc[random_int].to_dict())
        if response == "" or response == "n":
            df.at[random_int, "like"] = -1
            df.at[random_int, "score"] += -1
            print(f"You disliked {display_content_dict['video_id']} about {display_content_dict['content_category']}")
            print( df.iloc[random_int].to_dict())
            logs.append(df.iloc[random_int].to_dict())
        elif response == "q":
            break

    if display_content_dict["like"] == 1 or display_content_dict["like"] == -1:
        if input(f"Press q to quit") == "q":
            break


You liked video_56 about dog
{'video_id': 'video_56', 'content_category': 'dog', 'like': 1, 'score': 1}
You liked video_73 about dog
{'video_id': 'video_73', 'content_category': 'dog', 'like': 1, 'score': 1}
You liked video_33 about dog
{'video_id': 'video_33', 'content_category': 'dog', 'like': 1, 'score': 1}
You liked video_28 about space
{'video_id': 'video_28', 'content_category': 'space', 'like': 1, 'score': 1}
You liked video_72 about dog
{'video_id': 'video_72', 'content_category': 'dog', 'like': 1, 'score': 1}
You disliked video_17 about cat
{'video_id': 'video_17', 'content_category': 'cat', 'like': -1, 'score': -1}
You disliked video_7 about dog
{'video_id': 'video_7', 'content_category': 'dog', 'like': -1, 'score': -1}
You disliked video_61 about dog
{'video_id': 'video_61', 'content_category': 'dog', 'like': -1, 'score': -1}
You disliked video_90 about space
{'video_id': 'video_90', 'content_category': 'space', 'like': -1, 'score': -1}
You disliked video_70 about cat
{'vide

In [109]:
logs_df = pd.DataFrame(data=logs)
logs_df

,video_id,content_category,like,score
0,video_56,dog,1,1
1,video_73,dog,1,1
2,video_33,dog,1,1
3,video_28,space,1,1
4,video_72,dog,1,1
5,video_17,cat,-1,-1
6,video_7,dog,-1,-1
7,video_61,dog,-1,-1
8,video_90,space,-1,-1
9,video_70,cat,-1,-1


In [113]:
logs_df.groupby("content_category").describe()

like                                              score  \
                 count      mean       std  min  25%  50%  75%  max count   
content_category                                                            
cat                3.0 -0.333333  1.154701 -1.0 -1.0 -1.0  0.0  1.0   3.0   
dog                7.0  0.428571  0.975900 -1.0  0.0  1.0  1.0  1.0   7.0   
space              2.0  0.000000  1.414214 -1.0 -0.5  0.0  0.5  1.0   2.0   

                                                               
                      mean       std  min  25%  50%  75%  max  
content_category                                               
cat              -0.333333  1.154701 -1.0 -1.0 -1.0  0.0  1.0  
dog               0.428571  0.975900 -1.0  0.0  1.0  1.0  1.0  
space             0.000000  1.414214 -1.0 -0.5  0.0  0.5  1.0

In [115]:
# Here you can see, the user interacted with 11 videos out of which 3 were cat videos, 7 were dog videos, and 2 were space related videos. 
# Lets take a look at mean of all to get an idea of what user feels about the content categories.
# The user likes the dog content most, followed by neutral opinions about space, and then at last cat. 
# what can we infer from this? user is interested in dog content most, then space and then cat.
# that means the algorithm should show more dog content, a bit of space related content and then cat every now and then because user did like one video right?

### Lets see the same thing at scale, with more topics/content_categories, and more interaction methods such as time spent, sharing to friends, commenting, etc.
# **Slightly Better Algorithm**